# EGamma short exercise

Photons and electrons play a crucial role in the success of CMS. They were vital in the discovery of the Higgs boson and play an important role in searches for new physics and standard model measurements. The CMS detector features a high-resolution electromagnetic calorimeter (ECAL) and the world's largest active silicon tracker. These detectors provide clean object identification and excellent energy and momentum resolution of photons and electrons. In this exercise, we will examine photon and electron identification and study how these objects are used in physics analyses. We will start with [NanoAOD](https://twiki.cern.ch/twiki/bin/view/CMSPublic/WorkBookNanoAOD) samples.

## Finding samples
Let's find two simulation samples to use for the remainder of the exercise: a $Z\rightarrow \ell^{+}\ell^{-}$ sample, which will contain two electrons in slightly more than 1/3 of the events (Q: why > 1/3?); and a sample of $\gamma+\text{jets}$ events, where a photon is radiated off either initial- or final-state quarks.  To find samples via a web browser, one can go to the [DAS client](https://cmsweb.cern.ch/das/) page.  Here, we use the `dasgoclient` command-line utility, using Jupyter notebook's shell execution shortcut.

In [11]:
!dasgoclient --query="dataset dataset=/DYJetsToLL_M-50_Tune*/*NanoAODv5*/NANOAODSIM"

/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIAutumn18NanoAODv5-Nano1June2019_102X_upgrade2018_realistic_v19-v1/NANOAODSIM
/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIAutumn18NanoAODv5-Nano1June2019_102X_upgrade2018_realistic_v19_ext2-v1/NANOAODSIM
/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIFall17NanoAODv5-PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/NANOAODSIM
/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIFall17NanoAODv5-PU2017_12Apr2018_Nano1June2019_new_pmx_102X_mc2017_realistic_v7-v1/NANOAODSIM
/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIFall17NanoAODv5-PU2017_12Apr2018_Nano1June2019_new_pmx_102X_mc2017_realistic_v7_ext1-v1/NANOAODSIM
/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv5-Nano1June2019_102X_upgrade2018_realistic_v19-v1/NANOAODSIM
/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIFall17NanoAODv5-PU2017RECOSIMstep_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/NA

## Discriminating variables
   * Shower Shape Variable, sigmaIetaIeta: The energy weighted standard deviation of single crystal eta within the 5 × 5 crystals centered at the crystal with maximum energy.
    <img src="SWGuideCMSDataAnalysisSchoolLPC2019egamma/SigmaIEtaIEta_definition.png" alt="SigmaIEtaIEta_definition.png" width='400'  /> 
where the sum runs over the 5 × 5 crystal matrix around the most energetic crystal in the SuperCluster, and the eta distances are measured in units of the crystal size in the eta direction. This variable represents the second moment of the energy distribution along the eta coordinate.

   * R9: Ratio of the energy deposited in ECAL inside a 3 × 3 crystal matrix centered on the most energetic crystal in the supercluster and the superCluster raw energy.
   * H/E: Ratio of the energy deposited in the single closest HCAL tower to the supercluster position inside a cone of radius 0.15 in the eta - phi plane centered on the photon direction and the energy deposited in ECAL to that supercluster.
   * PF Photon Isolation (IPh): Sum of Et of all PF photons which are not in the “footprint” of the candidate photon and are within a cone of DeltaR = 0.3.
   * PF Charged Hadron Isolation (ICh ): Sum of Pt of all PF charged hadrons which are associated to the primary vertex, not in the “footprint” of the candidate photon, and within a cone of DeltaR = 0.3.
   * PF Neutral Hadron Isolation (INh ): Sum of Et of all PF neutral hadrons which are not in the “footprint” of the candidate photon and are within a cone of DeltaR = 0.3.
   * Conversion safe electron veto : This cut replaces the detector based pixel seed used to distinguish between photon and electron. It checks the DeltaR distance between the photon direction and the nearest electron track and remove photons if there is a GsfElectron matching to the photon supercluster, with no missing hits and having no matching reconstructed conversion.
   * Electron mini isolation: Uses a cone whose size varies as a function of the electron pT in order to reduce accidental overlap from hard scatter or pileup processes.

Note: The photon isolation cone is also susceptible to pileup from interactions not corresponding to the primary vertex. The correction from the pile-up energy contribution to the photon identification is applied using the following equation: Iso (new) = Iso (original) - (Rho_event *Effective Area), where Rho_event is the energy density computed using FastJet package. Rho_event is the median background density per unit area and a measure of the pile up activity in the event. The effective area is defined as the ratio of the slope obtained from linearly fitting Iso(Nvtx ) to the one from linearly Jets fitting Rho_event(Nvtx). The values of effective area for all three isolation criteria (photon, neutral and charged) are provided by the EGamma POG separately for barrel and endcap regions.
